In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter
import re
from dataclasses import dataclass
import tkinter as tk
from tkinter import filedialog

In [13]:
# =========================================================
# 1) 시간 파싱
# =========================================================
class TimeParser:
    @staticmethod
    def parse_time_flexible(tstr):
        s = str(tstr).strip()
        s = s.replace("오전", "AM").replace("오후", "PM")

        if "-" in s:
            m = re.search(r"(\d{1,2}:\d{1,2}(:\d{1,2})?)", s)
            if m:
                s = m.group(1)
            else:
                return 0.0

        parts = s.split(":")

        if len(parts) == 2:
            h = int(parts[0]); m = int(parts[1]); sec = 0
        elif len(parts) == 3:
            h = int(parts[0]); m = int(parts[1]); sec = int(parts[2])
        else:
            return 0.0

        return h + m/60 + sec/3600

In [14]:
# =========================================================
# 2) 데이터셋 클래스
# =========================================================
class GrowthDataset:
    def __init__(self, path: str, name: str = ""):
        self.path = path
        self.name = name if name else path
        self.df = self._load(path)

    def _load(self, path):
        try:
            df = pd.read_csv(path, encoding="utf-8-sig")
        except UnicodeDecodeError:
            df = pd.read_csv(path, encoding="cp949")

        df = df[[df.columns[0], df.columns[1]]]
        df.columns = ["time", "od"]

        df["time_h"] = df["time"].apply(TimeParser.parse_time_flexible)
        df["ln_od"] = np.where(df["od"] > 0, np.log(df["od"]), np.nan)
        return df

In [15]:
# =========================================================
# 3) LogPhase 데이터 클래스
# =========================================================
@dataclass
class LogPhaseInfo:
    start_idx: int
    end_idx: int
    slope: float
    intercept: float
    r2: float
    slope_smooth: np.ndarray
    t_valid: np.ndarray
    candidate_mask: np.ndarray

In [21]:
# =========================================================
# 4) 성장 분석기 클래스
# =========================================================
class GrowthAnalyzer:
    def __init__(self, smooth_window=7, poly_order=2, min_od=0.12, min_abs_slope=0.02):
        self.smooth_window = smooth_window
        self.poly_order = poly_order
        self.min_od = min_od
        self.min_abs_slope = min_abs_slope

    def find_log_phase_v4(self, df):
        t = df["time_h"].values
        y = df["ln_od"].values
        od = df["od"].values

        valid_idx = ~np.isnan(y)
        t_valid = t[valid_idx]
        y_valid = y[valid_idx]
        od_valid = od[valid_idx]

        sw = self.smooth_window
        if sw >= len(y_valid):
            sw = len(y_valid) if len(y_valid) % 2 == 1 else len(y_valid)-1
        if sw < 3: sw = 3
        if sw % 2 == 0: sw += 1

        y_smooth = savgol_filter(y_valid, sw, self.poly_order)
        slope = np.gradient(y_smooth, t_valid)

        cond = (od_valid >= self.min_od) & (slope > 0) & (slope >= self.min_abs_slope)

        if not np.any(cond):
            raise ValueError("log-phase 후보 없음. 설정값 조정 필요.")

        slope_candidate = np.where(cond, slope, -np.inf)
        start_local = int(np.argmax(slope_candidate))

        left = start_local
        while left > 0 and cond[left-1]:
            left -= 1
        right = start_local
        while right < len(cond)-1 and cond[right+1]:
            right += 1

        valid_idx_arr = np.where(valid_idx)[0]
        start_idx = int(valid_idx_arr[left])
        end_idx = int(valid_idx_arr[right])

        x = t[start_idx:end_idx+1]
        y_f = y[start_idx:end_idx+1]

        mu, intercept = np.polyfit(x, y_f, 1)
        ss_res = np.sum((y_f - (mu * x + intercept))**2)
        ss_tot = np.sum((y_f - np.mean(y_f))**2)
        r2 = 1 - ss_res/ss_tot if ss_tot > 0 else 0

        return LogPhaseInfo(start_idx, end_idx, mu, intercept, r2, slope, t_valid, cond)

    @staticmethod
    def calc_growth_params(log_info):
        mu = log_info.slope
        g = np.log(2) / mu
        return mu, g

    @staticmethod
    def growth_inhibition(mu_control, mu_treated):
        return (1 - mu_treated / mu_control) * 100

In [17]:
# =========================================================
# 5) 시각화 클래스
# =========================================================
class GrowthVisualizer:
    def __init__(self):
        plt.rcParams['font.family'] = 'Malgun Gothic'
        plt.rcParams['axes.unicode_minus'] = False

    def plot_growth(self, cdf, tdf, clog, tlog):
        plt.figure(figsize=(8, 5))
        plt.plot(cdf["time_h"], cdf["od"], marker="o", label="대조군")
        plt.plot(tdf["time_h"], tdf["od"], marker="s", label="처리군")

        c1, c2 = clog.start_idx, clog.end_idx
        t1, t2 = tlog.start_idx, tlog.end_idx

        plt.plot(cdf["time_h"].iloc[c1:c2+1], cdf["od"].iloc[c1:c2+1],
                 linewidth=3, label="대조군 log-phase")

        plt.plot(tdf["time_h"].iloc[t1:t2+1], tdf["od"].iloc[t1:t2+1],
                 linewidth=3, linestyle="--", label="처리군 log-phase")

        plt.xlabel("시간 (h)")
        plt.ylabel("OD")
        plt.title("E.coli Growth Curve")
        plt.grid()
        plt.legend()
        plt.show()

    def plot_ln(self, cdf, tdf):
        plt.figure(figsize=(8, 5))
        plt.plot(cdf["time_h"], cdf["ln_od"], marker="o", label="대조군 ln(OD)")
        plt.plot(tdf["time_h"], tdf["ln_od"], marker="s", label="처리군 ln(OD)")
        plt.xlabel("시간 (h)")
        plt.ylabel("ln(OD)")
        plt.grid()
        plt.legend()
        plt.show()

In [18]:
# =========================================================
# 6) 메인 Controller 클래스 (메뉴 담당)
# =========================================================
class GrowthApp:
    def __init__(self):
        self.control = None
        self.treated = None
        self.analyzer = GrowthAnalyzer()
        self.visual = GrowthVisualizer()
        self.control_log = None
        self.treated_log = None
        self.mu_control = None
        self.mu_treated = None
        self.g_control = None
        self.g_treated = None
        self.inhibition = None

    # ------------------------------------------------------
    # 파일 선택 (tkinter)
    # ------------------------------------------------------
    def file_select(self, title):
        root = tk.Tk()
        root.withdraw()

        # 👉 파일 선택창을 최상단으로 띄우는 핵심 코드
        root.attributes('-topmost', True)

        filepath = filedialog.askopenfilename(title=title)

        root.destroy()
        return filepath

    # ------------------------------------------------------
    # 초기 파일 2개 받기
    # ------------------------------------------------------
    def load_files(self):
        print("대조군 파일을 선택하세요.")
        cpath = self.file_select("대조군 CSV 선택")
        print("→ 선택됨:", cpath)

        print("\n처리군 파일을 선택하세요.")
        tpath = self.file_select("처리군 CSV 선택")
        print("→ 선택됨:", tpath)

        self.control = GrowthDataset(cpath, "대조군")
        self.treated = GrowthDataset(tpath, "처리군")

        print("\n[파일 로드 완료]\n")

    # ------------------------------------------------------
    # 분석 실행
    # ------------------------------------------------------
    def analyze(self):
        self.control_log = self.analyzer.find_log_phase_v4(self.control.df)
        self.treated_log = self.analyzer.find_log_phase_v4(self.treated.df)

        self.mu_control, self.g_control = self.analyzer.calc_growth_params(self.control_log)
        self.mu_treated, self.g_treated = self.analyzer.calc_growth_params(self.treated_log)

        self.inhibition = self.analyzer.growth_inhibition(self.mu_control, self.mu_treated)

        print("\n=== 분석 완료 ===")
        print(f"[대조군] μ={self.mu_control:.4f}, g={self.g_control:.4f} h")
        print(f"[처리군] μ={self.mu_treated:.4f}, g={self.g_treated:.4f} h")
        print(f"생장 저하율: {self.inhibition:.2f}%\n")

    # ------------------------------------------------------
    # 메뉴 루프
    # ------------------------------------------------------
    def menu(self):
        while True:
            print("\n=================================")
            print("       E.coli 성장 분석 프로그램")
            print("=================================")
            print("1. 성장 분석 실행")
            print("2. 성장 곡선(OD) 시각화")
            print("3. ln(OD) 그래프 시각화")
            print("4. 전체 실행(run_all)")
            print("0. 종료")
            print("=================================")

            choice = input("번호 입력: ")

            if choice == "1":
                self.analyze()

            elif choice == "2":
                if self.control_log is None:
                    print("※ 먼저 1번 분석을 실행하세요.")
                else:
                    self.visual.plot_growth(self.control.df, self.treated.df,
                                            self.control_log, self.treated_log)

            elif choice == "3":
                self.visual.plot_ln(self.control.df, self.treated.df)

            elif choice == "4":
                self.analyze()
                self.visual.plot_growth(self.control.df, self.treated.df,
                                        self.control_log, self.treated_log)
                self.visual.plot_ln(self.control.df, self.treated.df)

            elif choice == "0":
                print("프로그램을 종료합니다.")
                break

            else:
                print("잘못된 입력입니다.")

    # ------------------------------------------------------
    # 전체 실행 흐름
    # ------------------------------------------------------
    def run(self):
        self.load_files()      # 1) 파일 두 개 선택
        self.menu()            # 2) 메뉴 루프 시작

In [23]:
# =========================================================
# 실행 구문
# =========================================================
class Main():
    if __name__ == "__main__":
        app = GrowthApp()
        app.run()

대조군 파일을 선택하세요.
→ 선택됨: C:/Users/jangy/Desktop/JJJ/growth.csv

처리군 파일을 선택하세요.
→ 선택됨: C:/Users/jangy/Desktop/JJJ/OD값(std).csv

[파일 로드 완료]


       E.coli 성장 분석 프로그램
1. 성장 분석 실행
2. 성장 곡선(OD) 시각화
3. ln(OD) 그래프 시각화
4. 전체 실행(run_all)
0. 종료


번호 입력:  0


프로그램을 종료합니다.
